In [25]:
using Random

include("../test_data.jl")
include("../gnm_utils.jl")
include("../gnm.jl")

Random.seed!(1234)

TaskLocalRNG()

In [26]:
# load synthetic data
W_Y, D, A_init = load_weight_test_data()
A_Y = Float64.(W_Y .> 0);

In [24]:
limits_one=[-7.0, 7.0]
limits_two=[-7.0, 7.0]
limits_three=[0.02, 0.1]
limits_four=[0.8, 1.0]

n_runs = 10000

η = range(limits_one[1], stop=limits_one[2], length=max(floor(Int64, (n_runs)^(1/4)), 2))
γ = range(limits_two[1], stop=limits_two[2], length=max(floor(Int64, (n_runs)^(1/4)), 2))
α = range(limits_two[1], stop=limits_two[2], length=max(floor(Int64, (n_runs)^(1/4)), 2))
ω = range(limits_two[1], stop=limits_two[2], length=max(floor(Int64, (n_runs)^(1/4)), 2))

params_tups = unique(collect(Iterators.product(η, γ, α, ω)), dims=1)
params_tups = vec(permutedims(params_tups, [4, 3, 2, 1]))

params = zeros(length(params_tups), 4)
for iParams in 1:length(params_tups)
    params[iParams, 1] = params_tups[iParams][1]
    params[iParams, 2] = params_tups[iParams][2]
    params[iParams, 3] = params_tups[iParams][3]
    params[iParams, 4] = params_tups[iParams][4]
end

In [28]:
model = GNM_Mod.GNM(A_Y, D, A_init, params, 1, true, W_Y, 0)
GNM_Mod.generate_models(model)

InterruptException: InterruptException:

In [14]:
size(params_tups)

(31, 31, 31, 31)